## Read Me ##
The purpose of this script is to take in a list of NACCIDs in order to locate them in the UDS and MDS 
Phenotype files. The relevant data for each subject is extracted and the variables are harmonized into the ADGC phenotype format outlined in the appropriate data dictionary. Differences in the variable format between the UDS and MDS files creates complications that require documented conversions to ensure there is no confusion and that
any possible errors can be tracked.

This script requires several input, including the appropriate version of the MDS, UDS, and APOE files as well as the list of relevant NACCIDs for extraction.

In [ ]:
## Loading in Files ##
MDS_file = input("MDS File: ")
UDS_file = input("UDS File: ")
APOE_file = input("APOE File: ")
NACCID_file = input("NACCID File: ")

In [ ]:
import pandas as pd 
import numpy as np

#specifying extracted columns
MDS_cols = ['naccid','casecon','sex','birthyr','educ','race','hispanic','agedem',
'npdage','npbraak']
UDS_cols = ['NACCID','AUTOPSYCS','PHASE1CS','CS','SEX','BIRTHYR','EDUC',
'RACE','HISPANIC','DECAGE','LVISAGE','NACCDAGE','NACCBRAA']
APOE_cols = ['NACCID', 'APOE']

#assigning each file to a dataframe
MDS = pd.read_excel(MDS_file,usecols=MDS_cols,dtype=str)
UDS = pd.read_excel(UDS_file,usecols=UDS_cols,dtype=str)
APOE = pd.read_excel(APOE_file,usecols=APOE_cols,dtype={'APOE':str})
NACCID = pd.read_csv(NACCID_file,sep=' ',usecols=[1],header=None)

## Mapping Variables Across Files ##
phenotypes = [          # MDS:UDS:NEW conversions
    'NACCID',           # [0] same IDs
    'Sex',              # [1] 1:1:1 = Male, 2:2:2 = Female
    'DX',               # [2] 0::1 = Control, 1::2 = Case, ::3 = MCI, ::NA = NA
    'Age_at_onset',     # [3] for cases | agedem | DECAGE | NA:888
    'Age_at_death',     # [4] for autopsies | npdage | NACCDAGE
    'Age_last_visit',   # [5] for controls | age at death for MDS | LVISAGE
    'APOE',             # [6] APOE Genotype | 22, 23, 24, 33, 34, 44 | convert to standardize
    'Race',             # [7] 
    'Ethnicity',        # [8] 1:1:1 = Hispanic/Latino, 2:2:0 = Not, 9:9:NA = Not Applicable 
    'Notes',            # [9] In order to indicate anything abnormal about the subject's data
]
#reassessing column names for each variable mapping
fam.rename(columns={fam.columns[0]:phenotypes[0]},inplace=True)

MDS.rename(columns={'naccid':phenotypes[0],'sex':phenotypes[1],'casecon':phenotypes[2],
'race':phenotypes[7],'hispanic':phenotypes[8],'agedem':phenotypes[3],
'npdage':phenotypes[4]},inplace=True)

UDS.rename(columns={'SEX':phenotypes[1],'HISPANIC':phenotypes[8],'RACE':phenotypes[7],
'DECAGE':phenotypes[3],'LVISAGE':phenotypes[5],'NACCDAGE':phenotypes[4]},inplace=True)

# 'Race' | 3:3:3 = American Indian/Alaska Native, 4:5:4 = Asian, NA:4:4 = Native Hawaiian/Other PI,
#  2:2:2 = Black/African American, 1:1:1 = White, 50:50:50 = Other, 99:99:NA = Not Applicable
MDS[phenotypes[7]].replace({'99':'NA'},inplace=True)
UDS[phenotypes[7]].replace({'5':'4','99':'NA'},inplace=True)

# 'Ethnicity' | 1:1:1 = Hispanic/Latino, 2:2:0 = Not, 9:9:NA = Not Applicable 
MDS[phenotypes[8]].replace({'2':'0','9':'NA'},inplace=True)
UDS[phenotypes[8]].replace({'2':'0','9':'NA'},inplace=True)

# 'DX' / 'AD' status
# AUTOPSYCS | 2:1 = Control, 1:2 = Case | only UDS
UDS['AUTOPSYCS'].replace({'2':'1','1':'2'},inplace=True)  
# PHASE1CS | 2:1 = Control, 1:2 = Case | only UDS
UDS['PHASE1CS'].replace({'2':'1','1':'2'},inplace=True)
# CS | 1:1 = Control, 2:2 (probable AD status; assume NA (?)), 3:3 = MCI, 9:NA
UDS['CS'].replace({'9':'NA'},inplace=True)
# MDS 'casecon' | 0:1 = Control, 1:2 = Case
MDS[phenotypes[2]].replace({'0':'1','1':'2'},inplace=True)

#'Age' | populating Age_last_visit in MDS with Age_at_death
MDS[phenotypes[5]] = MDS[phenotypes[4]]

# 'Age' | NA:888
MDS[phenotypes[3]].fillna('888',inplace=True)
MDS[phenotypes[4]].fillna('888',inplace=True)
MDS[phenotypes[5]].fillna('888',inplace=True) #Age at death
UDS[phenotypes[3]].fillna('888',inplace=True)
UDS[phenotypes[4]].fillna('888',inplace=True)
UDS[phenotypes[5]].fillna('888',inplace=True)

## Creating variables given particular conditions ##
# DX / AD status | MDS avail | UDS conditional (AUTOPSYCS,PHASE1CS,CS)
UDS_AD = UDS['AUTOPSYCS'].fillna(UDS['PHASE1CS']).fillna(UDS['CS']).values
UDS.insert(loc=2,column=phenotypes[2],value=UDS_AD)

## Dropping unwanted columns ##
#MDS | 'birthyr', 'educ'
#UDS | 'BIRTHYR', 'EDUC'
MDS.drop(['birthyr','educ'],axis=1,inplace=True)
UDS.drop(['BIRTHYR', 'EDUC'],axis=1,inplace=True)
UDS.drop(['PHASE1CS', 'CS', 'AUTOPSYCS'],axis=1,inplace=True)

## Matching NACCIDS from fam to Phenotype Files ##
fam_MDS = pd.merge(fam,MDS,on="NACCID",how="left")
fam_UDS = pd.merge(fam,UDS,on="NACCID",how="left")
fam_APOE = pd.merge(fam,APOE,on="NACCID",how="left")
# .dropna()
fam_MDS.dropna(thresh=2,inplace=True)
fam_UDS.dropna(thresh=2,inplace=True)

# putting all together
final_df = fam_MDS.merge(fam_UDS,how='outer')
final_df = final_df.merge(fam_APOE,how='outer',on='NACCID')

## Exporting to Excel sheet ##
file_name = "ADC12_phenos.xlsx"
final_df_2 = final_df[['NACCID','Sex','DX','Age_at_onset','Age_at_death',
'Age_last_visit','APOE','Race','Ethnicity']]
final_df_2.to_excel(file_name,index=False)
